In [27]:
import nltk
import time
from nltk.corpus import stopwords
import nltk.data
import torch
import pdfplumber
from sentence_transformers import SentenceTransformer, util

In [2]:
# Download NLTK data (if not already downloaded)
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Error loading stopwords: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     self signed certificate in certificate chain
[nltk_data]     (_ssl.c:997)>
[nltk_data] Error loading punkt: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     self signed certificate in certificate chain
[nltk_data]     (_ssl.c:997)>
[nltk_data] Error loading wordnet: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     self signed certificate in certificate chain
[nltk_data]     (_ssl.c:997)>


False

In [28]:
# Load BERT models
model1 = SentenceTransformer('paraphrase-MiniLM-L3-v2')
model2 = SentenceTransformer('all-distilroberta-v1')
model3 = SentenceTransformer('multi-qa-distilbert-cos-v1')

In [29]:

def model(user_ques_1, user_ques_2):
    # Stop word removal
    stop = stopwords.words('english')
    ques_1_nstop = ' '.join([word for word in user_ques_1.split() if word not in stop])
    ques_2_nstop = ' '.join([word for word in user_ques_2.split() if word not in stop])

    # Tokenize the questions
    tok_ques_1 = nltk.word_tokenize(ques_1_nstop)
    tok_ques_2 = nltk.word_tokenize(ques_2_nstop)
    tok_ques_1_str = ' '.join(map(str, tok_ques_1))
    tok_ques_2_str = ' '.join(map(str, tok_ques_2))

    # Lemmatize data
    w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
    lemmatizer = nltk.stem.WordNetLemmatizer()

    def lemmatize_text(text):
        return ' '.join([lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(text)])

    ques_1_lemm_str = lemmatize_text(tok_ques_1_str)
    ques_2_lemm_str = lemmatize_text(tok_ques_2_str)

    # Convert lemmatized data to lower case
    ques_1_lemm_str = ques_1_lemm_str.lower()
    ques_2_lemm_str = ques_2_lemm_str.lower()

    # Text similarity scores obtained using 'paraphrase-MiniLM-L3-v2' BERT model
    st = time.time()
    embd1 = model1.encode(ques_1_lemm_str, convert_to_tensor=True)
    embd2 = model1.encode(ques_2_lemm_str, convert_to_tensor=True)
    cosine_scores1 = util.pytorch_cos_sim(embd1, embd2)
    et = time.time()
    elapsed_time = et - st

    # Text similarity scores obtained using BERT model 'all-distilroberta-v1'
    st = time.time()
    embd1 = model2.encode(ques_1_lemm_str, convert_to_tensor=True)
    embd2 = model2.encode(ques_2_lemm_str, convert_to_tensor=True)
    cosine_scores2 = util.pytorch_cos_sim(embd1, embd2)
    et = time.time()
    elapsed_time = et - st

    # Text similarity scores obtained using BERT model 'multi-qa-distilbert-cos-v1'
    st = time.time()
    embd1 = model3.encode(ques_1_lemm_str, convert_to_tensor=True)
    embd2 = model3.encode(ques_2_lemm_str, convert_to_tensor=True)
    cosine_scores3 = util.pytorch_cos_sim(embd1, embd2)
    et = time.time()
    elapsed_time = et - st

    # Calculate average similarity score
    avg_similarity_score = (cosine_scores1.item() + cosine_scores2.item() + cosine_scores3.item()) / 3

    return avg_similarity_score


In [30]:
# Define a function to extract text from a PDF file
def extract_text_from_pdf(pdf_path):
    with pdfplumber.open(pdf_path) as pdf:
        text = ""
        for page in pdf.pages:
            text += page.extract_text()
    return text

In [6]:
C:\Users\shric\Downloads\same_q.pdf
C:\Users\shric\Downloads\same_q2.pdf
    
C:\Users\shric\Desktop\Seminar\partial_same_1.pdf
C:\Users\shric\Desktop\Seminar\partial_same_2.pdf
    
    
C:\Users\shric\Desktop\Seminar\test1.pdf
C:\Users\shric\Desktop\Seminar\test2.pdf

SyntaxError: unexpected character after line continuation character (235552375.py, line 1)

In [45]:
# Prompt the user to enter the paths to the PDF files
pdf_path1 = input("Enter the path to the first PDF file: ")
pdf_path2 = input("Enter the path to the second PDF file: ")


# Extract text from both PDF files
question_paper1_text = extract_text_from_pdf(pdf_path1)
question_paper2_text = extract_text_from_pdf(pdf_path2)

Enter the path to the first PDF file: C:\Users\shric\Desktop\Seminar\partial_same_1.pdf
Enter the path to the second PDF file: C:\Users\shric\Desktop\Seminar\partial_same_2.pdf


In [46]:
print(question_paper1_text)
print("-----------------------------------------------------------")
print(question_paper2_text)

1. What is the difference between Python 2 and Python 3, and why is it important to migrate to
Python 3?
2. How can you create a Python func(cid:415)on that accepts a variable number of arguments or
keyword arguments?
3. Explain the Global Interpreter Lock (GIL) in Python. How does it impact mul(cid:415)-threading in
Python programs?
4. What is the purpose of virtual environments in Python, and how do you create and ac(cid:415)vate a
virtual environment using the standard library tools?
-----------------------------------------------------------
1. What is the difference between Python 2 and Python 3, and why is it important to migrate to
Python 3?
2. How can you create a Python func(cid:415)on that accepts a variable number of arguments or
keyword arguments?
3. What were the major causes and consequences of the American Civil War in the 19th
century?
4. How did the Industrial Revolu(cid:415)on in the 18th and 19th centuries transform economies,
socie(cid:415)es, and technology in West

In [47]:
#question_paper1_text = "1. Define and explain Unified Modeling Language (UML) 2. Explain conceptual model of UML 3. Explain building blocks in UML. 4. Explain things and relationships in UML. 5. List and explain diagrams in UML. 6. Explain common mechanism in UML"

#question_paper2_text = "1. Define and explain Unified Modeling Language (UML) 2. Explain conceptual model of UML 3. Explain building blocks in UML. 4. .List and explain UML2.0 diagrams. 5. Explain object diagram in UML."

import re

# Use regular expressions to split the text into individual questions
arr1 = re.split(r'\d+\.', question_paper1_text)
arr2 = re.split(r'\d+\.', question_paper2_text)

# Remove any leading or trailing whitespace from the questions
arr1 = [question.strip() for question in arr1 if question.strip()]
arr2 = [question.strip() for question in arr2 if question.strip()]

# Create an array to store the questions
arr1 = arr1
arr2 = arr2

print("arr1:", arr1)
print("--------------------------------------------------------------------------------------------")
print("arr2:", arr2)

new_paper = []
# flag = 0

for q1 in arr1:
    for q2 in arr2:
        similarity = model(q1, q2)
        if similarity > 0.7:
            print(similarity)
            new_paper.append(q1)
            
            break  # Break to avoid adding duplicates

# Add questions from question_paper2 that didn't have matches
for q2 in arr2:
    if q2 not in new_paper:
        new_paper.append(q2)
        
for q1 in arr1:
    if q1 not in new_paper:
        new_paper.append(q1)
print("--------------------------------------------------------------------------------------------")
# Print the new_paper array
print("new_paper:", new_paper)

arr1: ['What is the difference between Python 2 and Python 3, and why is it important to migrate to\nPython 3?', 'How can you create a Python func(cid:415)on that accepts a variable number of arguments or\nkeyword arguments?', 'Explain the Global Interpreter Lock (GIL) in Python. How does it impact mul(cid:415)-threading in\nPython programs?', 'What is the purpose of virtual environments in Python, and how do you create and ac(cid:415)vate a\nvirtual environment using the standard library tools?']
--------------------------------------------------------------------------------------------
arr2: ['What is the difference between Python 2 and Python 3, and why is it important to migrate to\nPython 3?', 'How can you create a Python func(cid:415)on that accepts a variable number of arguments or\nkeyword arguments?', 'What were the major causes and consequences of the American Civil War in the 19th\ncentury?', 'How did the Industrial Revolu(cid:415)on in the 18th and 19th centuries transform

In [ ]:
!pip install fitz
!pip install PyMuPDF


In [48]:
from reportlab.lib.pagesizes import A4
from reportlab.pdfgen import canvas


# Create a PDF document with A4 page size
pdf_file = "output.pdf"
c = canvas.Canvas(pdf_file, pagesize=A4)

# Set font and other attributes
c.setFont("Helvetica", 12)
line_height = 14  # Adjust line height as needed
bottom_margin = 20  # Margin to leave at the bottom of the page

# Calculate the available height on the page
page_height = A4[1] - bottom_margin

# Initialize page count
page_number = 1

# Iterate through the list and add the content to the PDF with pagination
y_position = page_height

for index, content in enumerate(new_paper, start=1):
    if y_position - line_height < 0:
        # If the content would overflow, start a new page
        c.showPage()
        c.setFont("Helvetica", 12)  # Reset font for the new page
        y_position = page_height
        page_number += 1

    content_with_index = f"{index}. {content}"
    c.drawString(20, y_position, content_with_index)
    y_position -= line_height

# Save the PDF document
c.save()


In [43]:
# Create a text file to write the content
txt_file = "output.txt"

# Open the text file for writing
with open(txt_file, "w") as file:
    for index, content in enumerate(new_paper, start=1):
        content_with_index = f"{index}. {content}\n"
        file.write(content_with_index)


In [49]:
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Paragraph
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.lib import colors
from reportlab.lib.enums import TA_CENTER
from reportlab.platypus import Spacer

output_file = 'output.pdf'

# Define custom page size with margins
page_width = letter[0]
page_height = letter[1]
left_margin = 36  # Adjust the margin values as needed
right_margin = 36
top_margin = 36
bottom_margin = 36

# Create a PDF document with custom margins
doc = SimpleDocTemplate(output_file, pagesize=(page_width, page_height), leftMargin=left_margin, rightMargin=right_margin, topMargin=top_margin, bottomMargin=bottom_margin)

# Create a list to store the content of the PDF
story = []

# Define a style for the content
styles = getSampleStyleSheet()
style = styles['Normal']
style.alignment = TA_CENTER  # Center align the tex

# Open and read the input text file
with open(txt_file, 'r') as file:
    for line in file:
        paragraph = Paragraph(line.strip(), style)
        story.append(paragraph)
        story.append(Spacer(1, 5))  # Add some space between paragraphs

# Build the PDF document
doc.build(story)

print(f'PDF file "{output_file}" generated successfully.')


PDF file "output.pdf" generated successfully.
